In [ ]:
### Baseline pixel-wise classification with random forest. ###

library(raster)
library(rgdal)
library(randomForest)

randomforest_classifier <- function(training_path, data_path, out_path) {
  name <- strsplit(strsplit(data_path, split = '.tif')[[1]][[1]], split = '/')[[1]]
  filename <- name[[length(name)]]
  data <- stack(data_path)
  training <- readOGR(training_path)
  training_r <- rasterize(training, data[[1]], field = 'id')
  names(training_r) <- 'class'
  data_training <- mask(data, training_r)
  trainingbrick <- addLayer(data_training, training_r)
  valuetable <- getValues(trainingbrick)
  valuetable <- na.omit(valuetable)
  valuetable <- as.data.frame(valuetable)
  
  modelRF <- randomForest(x = valuetable[ ,c(1:length(data@layers))], y = valuetable$class, importance = TRUE, ntree = 500)
  classified <- predict(data, model = modelRF, na.rm = TRUE)
  writeRaster(classified, 'GTiff', filename = paste0(out_path, '/', filename, '_rf'), overwrite = T)
}

# training areas
roi <- ''
# Images that are to be classified
input <- ''
# Folder where the result is stored
outfolder <- ''
randomforest_classifier(roi, input, outfolder)